In [12]:
%%time

import numpy as np
import torch
import gc
import random
from torch import nn
from sklearn.metrics import mean_absolute_error
from tqdm.notebook import tqdm
from NN_models import NN_2_256, NN_4_256, NN_8_256, NN_8_64, NN_4_128, NN_8_128
from reaction_energy_calculation import calculate_reaction_energy
from prepare_data import prepare, save_chk, load_chk


def set_random_seed(seed):
    # seed everything
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

set_random_seed(41)

CPU times: user 342 µs, sys: 61 µs, total: 403 µs
Wall time: 414 µs


In [13]:
# %%time
# data, data_train, data_test = prepare(path='data', test_size=0.2)

# %%time
# save_chk(data, data_train, data_test, path='checkpoints')

In [14]:
%%time
data, data_train, data_test = load_chk(path='checkpoints')

CPU times: user 4.7 s, sys: 3.77 s, total: 8.47 s
Wall time: 8.53 s


In [15]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data):

        self.data = data
        
    def __getitem__(self, i):
        self.data[i].pop('Database', None)

        return self.data[i], self.data[i]['Energy']
    
    def __len__(self):
        return len(self.data.keys())


train_set = Dataset(data=data_train)
train_dataloader = torch.utils.data.DataLoader(train_set, 
                                               batch_size=None,
                                               num_workers=1,
                                               pin_memory=True,
                                               shuffle=True)

test_set = Dataset(data=data_test)
test_dataloader = torch.utils.data.DataLoader(test_set, 
                                              batch_size=None,
                                              num_workers=1,
                                              pin_memory=True,
                                              shuffle=True)

In [30]:
device = torch.device('cuda:0') if torch.cuda.is_available else torch.device('cpu')
print(device)
criterion = nn.MSELoss()
model = NN_4_256(nconstants=24, DFT='PBE').to(device)

model.load_state_dict(torch.load('model_chk/predopt/PBE_4_256.param', map_location=device))

cuda:0


<All keys matched successfully>

In [37]:
from importlib import reload
import predopt
reload(predopt)
import NN_models
reload(NN_models)

<module 'NN_models' from '/home/duzaripov/ML-parameterization-of-DFT-functionals/NN_models.py'>

In [38]:
%%time

from predopt import DatasetPredopt
train_predopt_set = DatasetPredopt(data=data, dft='PBE')
train_predopt_dataloader = torch.utils.data.DataLoader(train_predopt_set,
                                                       batch_size=None,
                                                       num_workers=1,
                                                       pin_memory=True,
                                                       shuffle=True)

CPU times: user 215 µs, sys: 0 ns, total: 215 µs
Wall time: 225 µs


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-5, betas=(0.9, 0.999), weight_decay=0.01)

In [ ]:
from predopt import predopt

predopt(model, criterion, optimizer, train_predopt_dataloader, device, n_epochs=1, accum_iter=10)

Epoch 1


  0%|          | 0/267 [00:00<?, ?it/s]

train MSE Loss = 0.00020600
train MAE Loss = 0.00589636


([0.00020599788444400992], [0.0058963555])

Bad owner or permissions on /home/duzaripov/.ssh/config


In [42]:
# torch.save(model.state_dict(), 'model_chk/predopt/PBE_4_256.param')

In [ ]:
# !cp model_chk/predopt/PBE_4_256.param model_chk/predopt/PBE_4_256.param_backup3

In [43]:
torch.cuda.empty_cache()

In [44]:
from tqdm.notebook import tqdm
import os

log_file_path = 'log/epoch_training.log'

if os.path.isfile(log_file_path):
    os.remove(log_file_path)

In [45]:
def train(model, criterion, optimizer, train_loader, test_loader, n_epochs=20, accum_iter=4, verbose=False):
    train_loss_mae = []
    train_loss_mse = []
    test_loss_mae = []
    test_loss_mse = []

    for epoch in range(n_epochs):
        print(f'Epoch {epoch+1}')
        # train
        model.train()
        progress_bar_train = tqdm(train_loader)
        train_mae_losses_per_epoch = []
        train_mse_losses_per_epoch = []
        optimizer.zero_grad()
        for batch_idx, (X_batch, y_batch) in enumerate(progress_bar_train):
            
            X_batch_grid, y_batch = X_batch['Grid'].to(device), y_batch.to(device)
            predictions = model(X_batch_grid)
            reaction_energy = calculate_reaction_energy(X_batch, predictions, device, rung='GGA', dft='PBE')

            if verbose:
                print(f"{X_batch['Components']} pred {reaction_energy.item():4f} true {y_batch.item():4f}")
            loss = criterion(reaction_energy, y_batch)
            MSE = loss.item()
            MAE = torch.abs(reaction_energy - y_batch).item()
            print(MAE)
            train_mse_losses_per_epoch.append(MSE)
            train_mae_losses_per_epoch.append(MAE)
            progress_bar_train.set_postfix(MSE = MSE, MAE = MAE)
            
            with open(log_file_path, 'a') as f:
                f.write(f"{X_batch['Components']} pred {reaction_energy.item():4f} true {y_batch.item():4f} MSE {MSE:4f} MAE {MAE:4f}\n")
            
            # loss_accumulation
            loss = loss / accum_iter
            loss.backward()
            if ((batch_idx + 1) % accum_iter == 0) or (batch_idx + 1 == len(train_loader)):
                optimizer.step()
                optimizer.zero_grad()
                
            del X_batch, X_batch_grid, y_batch, predictions, reaction_energy
            gc.collect()
            torch.cuda.empty_cache()
            # gpu_usage()
        
        train_loss_mse.append(np.mean(train_mse_losses_per_epoch))        
        train_loss_mae.append(np.mean(train_mae_losses_per_epoch))

        print(f'train MSE Loss = {train_loss_mse[epoch]:.8f} MAE Loss = {train_loss_mae[epoch]:.8f}')


        
        #test
        model.eval()
        progress_bar_test = tqdm(test_loader)
        test_mae_losses_per_epoch = []
        test_mse_losses_per_epoch = []
        with torch.no_grad():
            for X_batch, y_batch in progress_bar_test:
                X_batch_grid, y_batch = X_batch['Grid'].to(device), y_batch.to(device)
                # print(f"{X_batch['Components']}")
                predictions = model(X_batch_grid)
                reaction_energy = calculate_reaction_energy(X_batch, predictions, device, rung='GGA', dft='PBE')
                loss = criterion(reaction_energy, y_batch)
                MSE = loss.item()
                MAE = torch.abs(reaction_energy - y_batch).item()
                test_mse_losses_per_epoch.append(MSE)
                test_mae_losses_per_epoch.append(MAE)
                progress_bar_test.set_postfix(MSE = MSE, MAE = MAE)

                
                with open(log_file_path, 'a') as f:
                    f.write(f"{X_batch['Components']} pred {reaction_energy.item():4f} true {y_batch.item():4f} MSE {MSE:4f} MAE {MAE:4f}\n")
                del X_batch, X_batch_grid, y_batch, predictions, reaction_energy, loss, MAE, MSE
                gc.collect()
                torch.cuda.empty_cache()
                
        test_loss_mse.append(np.mean(test_mse_losses_per_epoch))
        test_loss_mae.append(np.mean(test_mae_losses_per_epoch))

        print(f'test MSE Loss = {test_loss_mse[epoch]:.8f} MAE Loss = {test_loss_mae[epoch]:.8f}')

    return train_loss_mae

In [7]:
from importlib import reload
import PBE
import reaction_energy_calculation
import utils
import NN_models
reload(NN_models)
reload(utils)
reload(reaction_energy_calculation)
reload(PBE)

<module 'PBE' from '/home/duzaripov/ML-parameterization-of-DFT-functionals/PBE.py'>

In [9]:
model = NN_4_256(nconstants=24, DFT='PBE').to(device)
model.zero_grad()
# model.load_state_dict(torch.load('model_chk/predopt/PBE_4_256.param', map_location=device))

<All keys matched successfully>

In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
train_loss_mae = train(model, criterion, optimizer,
                       train_dataloader, test_dataloader,
                       n_epochs=1, accum_iter=5)

Epoch 1


  0%|          | 0/215 [00:00<?, ?it/s]

42.73004150390625
70.08631134033203
12.52762222290039
41.67515563964844
317.76318359375
c_arr is NaN


ValueError: NaN detected

In [ ]:
train_loss_mae

In [13]:
def test_dft(train_loader, test_loader, c_arr, criterion, rung, dft, verbose=False, n_epochs=1, model_eval=False):
    train_loss_mae = []
    train_loss_mse = []
    test_loss_mae = []
    test_loss_mse = []

    for epoch in range(n_epochs):
        print(f'Epoch {epoch+1}')
        # train

        progress_bar_train = tqdm(train_loader)
        train_mae_losses_per_epoch = []
        train_mse_losses_per_epoch = []
        for X_batch, y_batch in progress_bar_train:
            # print(f"{X_batch['Components']}")
            X_batch_grid, y_batch = X_batch['Grid'].to(device), y_batch.to(device)
            # print(torch.tile(c_arr, [X_batch_grid.shape[0],1]))
            if model_eval:
                predictions = model(X_batch_grid)
            else:
                predictions = torch.tile(c_arr, [X_batch_grid.shape[0],1]).to(device)
            reaction_energy = calculate_reaction_energy(X_batch, predictions, device, rung, dft)
            loss = criterion(reaction_energy, y_batch)
            MSE = loss.item()
            MAE = torch.abs(reaction_energy - y_batch).item()
            train_mse_losses_per_epoch.append(MSE)
            train_mae_losses_per_epoch.append(MAE)
            progress_bar_train.set_postfix(MSE = MSE, MAE = MAE)
            if verbose:
                print(f"{X_batch['Components']} pred {reaction_energy.item():4f} true {y_batch.item():4f} MSE {MSE:4f} MAE {MAE:4f}")

            del X_batch, X_batch_grid, y_batch, predictions, reaction_energy, loss, MAE, MSE
            gc.collect()
            torch.cuda.empty_cache()

        
        train_loss_mse.append(np.mean(train_mse_losses_per_epoch))        
        train_loss_mae.append(np.mean(train_mae_losses_per_epoch))

        print(f'train MSE Loss = {train_loss_mse[epoch]:.8f} MAE Loss = {train_loss_mae[epoch]:.8f}')


        
        #test
        progress_bar_test = tqdm(test_loader)
        test_mae_losses_per_epoch = []
        test_mse_losses_per_epoch = []
        for X_batch, y_batch in progress_bar_test:
            # print(f"{X_batch['Components']}")
            X_batch_grid, y_batch = X_batch['Grid'].to(device), y_batch.to(device)
            if model_eval:
                predictions = model(X_batch_grid)
            else:
                predictions = torch.tile(c_arr, [X_batch_grid.shape[0],1]).to(device)
            reaction_energy = calculate_reaction_energy(X_batch, predictions, device, rung, dft)
            loss = criterion(reaction_energy, y_batch)
            MSE = loss.item()
            MAE = torch.abs(reaction_energy - y_batch).item()
            # progress_bar_train.set_postfix(MAE = MAE)
            test_mse_losses_per_epoch.append(MSE)
            test_mae_losses_per_epoch.append(MAE)
            if verbose:
                print(f"{X_batch['Components']} pred {reaction_energy.item():4f} true {y_batch.item():4f} MSE {MSE:4f} MAE {MAE:4f}")
            
            del X_batch, X_batch_grid, y_batch, predictions, reaction_energy, loss, MAE, MSE
            gc.collect()
            torch.cuda.empty_cache()
                
        test_loss_mse.append(np.mean(test_mse_losses_per_epoch))
        test_loss_mae.append(np.mean(test_mae_losses_per_epoch))

        print(f'test MSE Loss = {test_loss_mse[epoch]:.8f} MAE Loss = {test_loss_mae[epoch]:.8f}')

    return train_loss_mae

In [14]:
from importlib import reload
import NN_models
reload(NN_models)
from NN_models import NN_2_256, NN_8_256, NN_8_64
# import SVWN3

In [15]:
from importlib import reload
import PBE
import reaction_energy_calculation
import utils
import NN_models
reload(NN_models)
reload(utils)
reload(reaction_energy_calculation)
reload(PBE)

<module 'PBE' from '/home/duzaripov/ML-parameterization-of-DFT-functionals/PBE.py'>

In [16]:
true_constants_SVWN = torch.Tensor([0.0310907, 0.01554535, 
                3.72744,   7.06042,
                12.9352,   18.0578,
                -0.10498,  -0.32500,
                0.0310907,  0.01554535,  -1/(6*torch.pi**2),
                13.0720,    20.1231,      1.06835,
                42.7198,   101.578,      11.4813,
                -0.409286,  -0.743294,   -0.228344,
                1])

true_constants_PBE = torch.Tensor([[0.06672455,
       (1 - torch.log(torch.Tensor([2])))/(torch.pi**2),
       1.709921,
       7.5957, 14.1189, 10.357,
       3.5876, 6.1977, 3.6231,
       1.6382, 3.3662,  0.88026,
       0.49294, 0.62517, 0.49671,
       # 1,  1,  1,
       0.031091, 0.015545, 0.016887,
       0.21370,  0.20548,  0.11125,
       -3/8*(3/torch.pi)**(1/3)*4**(2/3),
       0.8040,
       0.2195149727645171]])

par_rpw92_constants = torch.Tensor([[0.06672455,
       (1 - torch.log(torch.Tensor([2])))/(torch.pi**2),
       1.709920934161365617563962776245,
       7.5957, 14.1189, 10.357,
       3.5876, 6.1977, 3.6231,
       1.6382, 4.86059,  0.88026,
       0.49294, 0.750188, 0.49671,
       # 1,  1,  1,
       0.0310907, 0.01554535, 0.0168869,
       0.21370,  0.266529,  0.11125,
       -3/8*(3/torch.pi)**(1/3)*4**(2/3),
       0.8040,
       0.2195149727645171]])


# true_constants

test_dft(train_dataloader, test_dataloader, true_constants_PBE, criterion, rung='GGA', dft='PBE', verbose=False, model_eval=True)
# test_dft(train_dataloader, test_dataloader, true_constants_SVWN, criterion, rung='LDA', dft='SVWN3', verbose=False)

# broken_PBE 45.47375717 35.76542641
# Mean Absolute Error, kcal/mol
# DFT              train        test
# SVWN3            32.63222432 26.81526929
# PBE_sigma_broken 7.84055060  7.63013082
# PBE_fixed        7.85833597  7.64929252
# PBE_pred_4_256      

Epoch 1


  0%|          | 0/215 [00:00<?, ?it/s]

train MSE Loss = 11108.82772111 MAE Loss = 76.07116647


  0%|          | 0/52 [00:00<?, ?it/s]

test MSE Loss = 7573.58364240 MAE Loss = 65.99973547


[76.07116646517154]

In [16]:
from importlib import reload
import PBE
import reaction_energy_calculation
import utils
import NN_models
reload(NN_models)
reload(utils)
reload(reaction_energy_calculation)
reload(PBE)
from reaction_energy_calculation import calculate_reaction_energy

In [117]:
from importlib import reload
import PBE
reload(PBE)
# from PBE_new import F_PBE_new

<module 'PBE' from '/home/duzaripov/ML-parameterization-of-DFT-functionals/PBE.py'>

In [35]:
calculate_reaction_energy(data[73], torch.tile(true_constants_PBE, [data[73]['Grid'].shape[0],1]).to(device='cpu'), device='cpu', rung='GGA', dft='PBE')

tensor(104.6450347900390625000000000)